In [2]:
# Appendix A
# Shor's Quantum Factoring Algorithm

# Adapted from https://github.com/ttlion/ShorAlgQiskit/blob/master/Shor_Normal_QFT.py
# Based on Polynomial time algorithms for discrete logarithms and factoring on a quantum computer by P. Shor 1994

# Takes integer input from user which is the product of two primes
# Finds a random a where 1 < a < N and a is coprime with N
# Applies quantum phase estimation to find phase
# Gets r from phase = s/r
# Uses r to find factors of or numbers that share a factor with N

In [3]:
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ, BasicAer
from qiskit.compiler import transpile, assemble
from qiskit.providers.ibmq import least_busy
from qiskit.tools.jupyter import *
from qiskit.visualization import *

from random import randint
import math
from time import time
import numpy as np
import fractions
from fractions import Fraction

%matplotlib inline

In [4]:
# Returns modular inverse of a mod m
# Returns 1 otherwise
def modinv(a, m) : 
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

In [5]:
# Check if N is a perfect power
# Perfect power N = q^p
def check_if_power(N):
    b=2
    while (2**b) <= N:
        a = 1
        c = N
        while (c-a) >= 2:
            m = int( (a+c)/2 )

            if (m**b) < (N+1):
                p = int( (m**b) )
            else:
                p = int(N+1)

            if int(p) == int(N):
                print('N is {0}^{1}'.format(int(m),int(b)) )
                return True

            if p<N:
                a = int(m)
            else:
                c = int(m)
        b=b+1

    return False

In [6]:
# Returns a | 1 < a < N
def get_value_a(N):
    a = randint(2, N-1)
    
    while math.gcd(a,N)!=1:
        a = randint(2, N-1)
        
    return a

In [7]:
def get_angles(a, bitlen):
    s = bin(int(a))[2:].zfill(bitlen)
    angles = np.zeros(bitlen)
    for i in range(0, bitlen):
        for j in range(i,bitlen):
            if s[j] == '1':
                angles[bitlen-i-1]+=math.pow(2, -(j-i))
        angles[bitlen-i-1]*=np.pi
    return angles

In [8]:
# Doubly controlled U gate
def ccphase(qc, angle, ctl1, ctl2, tgt):
    qc.cu1(angle/2, ctl1, tgt)
    qc.cx(ctl2, ctl1)
    qc.cu1(-angle/2, ctl1, tgt)
    qc.cx(ctl2, ctl1)
    qc.cu1(angle, ctl2, tgt)

In [9]:
# Adds classical to quantum register
def phiADD(qc, q_reg, a, bitlen, inv):
    angles = get_angles(a, bitlen)
    for i in range(bitlen-1):
        if inv == 0:
            qc.u1(angles[i], q_reg[i])
        else:
            qc.u1(-angles[i], q_reg[i])

In [10]:
# Controlled phiADD gate
def cphiADD(qc, q_reg, ctl, a, bitlen, inv):
    angles = get_angles(a, bitlen)
    for i in range(bitlen):
        if inv == 0:
            qc.cu1(angles[i], ctl, q_reg[i])
        else:
            qc.cu1(-angles[i], ctl, q_reg[i])
    

In [11]:
# Doubly controlled phiADD gate
def ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, inv):
    angles = get_angles(a, bitlen)
    for i in range(bitlen):
        if inv == 0:
            ccphase(qc, angles[i], ctl1, ctl2, q_reg[i])
        else:
            ccphase(qc, -angles[i], ctl1, ctl2, q_reg[i])

In [12]:
# Applies QFT to specified quantum register 
# Will also swap qubits if specified
def qft(qc, q_reg, bitlen, swaps):
    i=bitlen-1
    # Apply the H gates and Cphases
    while i>=0:
        qc.h(q_reg[i])        
        j=i-1  
        while j>=0:
            if (np.pi)/(pow(2,(i-j))) > 0:
                qc.cu1( (np.pi)/(pow(2,(i-j))) , q_reg[i] , q_reg[j] )
                j=j-1   
        i=i-1 
        
    # Apply swaps
    if swaps==1:
        i=0
        while i < ((bitlen-1)/2):
            circuit.swap(q_reg[i], q_reg[bitlen-1-i])
            i=i+1

In [13]:
# Applies inverse QFT to specified quantum register
# Will swap qubits if specified
def qft_inv(qc, q_reg, bitlen, swaps):
    # If specified, apply the Swaps at the beginning
    if swaps==1:
        i=0
        while i < ((bitlen-1)/2):
            qc.swap(q_reg[i], q_reg[bitlen-1-i])
            i=i+1
    # Apply the H gates and Cphases
    i=0
    while i<bitlen:
        qc.h(q_reg[i])
        if i != bitlen-1:
            j=i+1
            y=i
            while y>=0:
                 if (np.pi)/(pow(2,(j-y))) > 0:
                    qc.cu1( - (np.pi)/(pow(2,(j-y))) , q_reg[j] , q_reg[y] )
                    y=y-1   
        i=i+1

In [14]:
# Doubly controlled phiADDmodN gate
# Adds a to q_reg only if ctl1 and ctl2 are | 1 >
# Then applies inverse phiADD to subtract N from the quantum register
def ccphiADDmodN(qc, q_reg, ctl1, ctl2, aux, a, N, bitlen):
    ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, 0)
    phiADD(qc, q_reg, N, bitlen, 1)
    qft_inv(qc, q_reg, bitlen, 0)
    qc.cx(q_reg[bitlen-1], aux)
    qft(qc, q_reg, bitlen, 0)
    cphiADD(qc, q_reg, aux, N, bitlen, 0)
    
    ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, 1)
    qft_inv(qc, q_reg, bitlen, 0)
    qc.x(q_reg[bitlen-1])
    qc.cx(q_reg[bitlen-1], aux)
    qc.x(q_reg[bitlen-1])
    qft(qc, q_reg, bitlen, 0)
    ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, 0)

In [15]:
# Inverse of above
# Will only apply if both ctl1 and ctl2 are | 1 >
def ccphiADDmodN_inv(qc, q_reg, ctl1, ctl2, aux, a, N, bitlen):
    ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, 1)
    qft_inv(qc, q_reg, bitlen, 0)
    qc.x(q_reg[bitlen-1])
    qc.cx(q_reg[bitlen-1],aux)
    qc.x(q_reg[bitlen-1])
    qft(qc, q_reg, bitlen, 0)
    ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, 0)
    cphiADD(qc, q_reg, aux, N, bitlen, 1)
    qft_inv(qc, q_reg, bitlen, 0)
    qc.cx(q_reg[bitlen-1], aux)
    qft(qc, q_reg, bitlen, 0)
    phiADD(qc, q_reg, N, bitlen, 0)
    ccphiADD(qc, q_reg, ctl1, ctl2, a, bitlen, 1)

In [16]:
# The U gate that is applied to the quantum register
def cMULTmodN(qc, ctl, q_reg, aux, a, N, bitlen):
    qft(qc, aux, bitlen+1, 0)
    
    for i in range(0, bitlen):
        ccphiADDmodN(qc, aux, q_reg[i], ctl, aux[bitlen+1], (2**i)*a % N, N, bitlen+1)
        
    qft_inv(qc, aux, bitlen+1, 0)
    
    for i in range(0, bitlen):
        qc.cswap(ctl, q_reg[i], aux[i])
        
    a_inv = modinv(a, N)
    
    qft(qc, aux, bitlen+1, 0)
    i = bitlen-1
    while i >= 0:
        ccphiADDmodN_inv(qc, aux, q_reg[i], ctl, aux[bitlen+1], math.pow(2,i)*a_inv % N, N, bitlen+1)
        i -= 1
    qft_inv(qc, aux, bitlen+1, 0)


In [17]:
# Initializes quantum circuit and applies quantum phase estimation
'''
                                     ______
     |0> -H------------ ... ---O----|      |-m-
     |0> -H-------O---- ... ---|----|      |-m-
| a >|0> -H-----O-|---- ... ---|----|      |-m-
                | |      .     |    |QFT^-1|
                | |      .     |    |      |
                | |      .     |    |      |
     |0> -H---O-|-|---- ... ---|----|      |-m-
     |0> -H-O-|-|-|---- ... ---|----|______|-m-
            | | | |            |
            | | | |            |
| b >|1> ---U-U-U-U---- ... ---U---------------

'''
# Circuit will look like the above once this function is called
def qpe(a, N, bitlen):
    
    # Auxilliary quantum register used in addition and multiplication
    aux = QuantumRegister(bitlen+2)
    
    # Quantum register where the sequential QFT is performed
    a_reg = QuantumRegister(2*bitlen)
    
    # Quantum register where the multiplications are made
    b_reg = QuantumRegister(bitlen)
    
    # Classical register where the measured values of the QFT are stored
    classic = ClassicalRegister(2*bitlen)

    # Quantum circuit initialization
    qc = QuantumCircuit(b_reg , a_reg , aux, classic)
    
    # Initialize down register to 1 and create maximal superposition in top register
    qc.h(a_reg)
    qc.x(b_reg[0])

    # Apply U gates
    for i in range(0, 2*bitlen):
        cMULTmodN(qc, a_reg[i], b_reg, aux, int(pow(a, pow(2, i))), N, bitlen)

    # Apply inverse QFT
    qft_inv(qc, a_reg, 2*bitlen, 1)

    # Measure top register to get value
    qc.measure(a_reg, classic)
    
    # Return circuit
    return qc

In [18]:
def shor(N):

    time_i = time()
    
    while True:
    
        # Step 1-3: 
        # Get random a, coprime with N
        # Determine if gcd is trivial
        
        a = get_value_a(N)

        # Get n value used in Shor's algorithm, to know how many qubits are used
        bitlen = math.ceil(math.log(N,2))

        # Create quantum circuit and apply QPE

        qc = qpe(a, N, bitlen)

        
        # Step 4
        # Once it is determined that QPE is necessary,
        # run simulation of circuit
        # Simulation will output phase = s/r
        

        device = provider.get_backend('ibmq_16_melbourne')
        backend = Aer.get_backend('qasm_simulator')
        simulation = execute(qc, backend, shots=1, memory=True)
        result = simulation.result()
        readings = result.get_memory()

        # Get period r from the fraction s/r
        
        phase = int(readings[0],2)/(2**bitlen)

        frac = Fraction(phase).limit_denominator(15)
        
        # Can skip continued fractions since python is able to split 
        # a decimal into its numerator and denominator
        
        s, r = frac.numerator, frac.denominator
        
        guesses = [math.gcd(a**(r//2)-1, N), math.gcd(a**(r//2)+1, N)]
        

        if (guesses[0] != 1 and guesses[0] != N):
            print("\n\tNon-trivial factors found: %i and %i" % (guesses[0], (N/guesses[0])))
            break
        elif (guesses[1] != 1 and guesses[1] != N):
            print("\n\tNon-trivial factors found: %i and %i" % (guesses[1], (N/guesses[1])))
            break
        else:
            print("\n\tFound trivial guesses %i and %i" % (guesses[0], guesses[1]))
            print("\tRestarting circuit...\n")
        
    
    time_f = time()
    time_delta = time_f - time_i
    
    print("\n\tAlgorithm took: %f seconds" % time_delta)
    return time_delta

In [19]:
if __name__ == '__main__':
    
    print("--------------------")
    print("Appendix A")
    print("Output for Shor's Quantum Factorization Algorithm\n\n")


    
    provider = IBMQ.load_account()

    # User enters number {N | N != 1, N != 0, N != q^p}

    while True:
        N = int(input("Please input integer number N: "))

        # Ask for N again if N equals 1 or 0

        if N==1 or N==0: 
            print("Please put an N different from 0 and from 1")
            continue

        # Also if N can be put in N=p^q, p>1, q>=2

        if check_if_power(N)==True:
            continue

        break
    
    # Initialize variables
    times = []
    runs = int(input("How many times to run: "))
    sums = 0
    
    # Runs factoring algorithm a number of times
    # Gets total amount of time
    # Saves individual time per run for later
    for i in range(runs):
        print("\n\n****************\nRun # %i" % (i+1))
        time_to_run = shor(N)
        sums += time_to_run
        times.append(time_to_run)
    
    # Average amount of time per run
    avg = sums/runs
    # Sum of deviations from average
    devsum = 0

    for item in times:
        # Calculates square of each deviation
        dev = (avg - item)**2
        # Stores sum of squares
        devsum += dev

    # Gets the square of the average of the deviations
    # Calculates standard deviation
    avg_sq = devsum/runs    
    standard_dev = avg_sq**(1/2) 
        
    
    print("\nAverage time per run: %f" % (avg))
    print("Standard Deviation: %f" % standard_dev)
    
    print("Done.")
    print("--------------------")


--------------------
Appendix A
Output for Shor's Quantum Factorization Algorithm


Please input integer number N: 21
How many times to run: 10


****************
Run # 1

	Non-trivial factors found: 3 and 7

	Algorithm took: 115.337805 seconds


****************
Run # 2

	Non-trivial factors found: 3 and 7

	Algorithm took: 114.771893 seconds


****************
Run # 3

	Non-trivial factors found: 3 and 7

	Algorithm took: 113.745415 seconds


****************
Run # 4

	Found trivial guesses 1 and 21
	Restarting circuit...


	Found trivial guesses 21 and 1
	Restarting circuit...


	Non-trivial factors found: 3 and 7

	Algorithm took: 354.691870 seconds


****************
Run # 5

	Found trivial guesses 21 and 1
	Restarting circuit...


	Non-trivial factors found: 3 and 7

	Algorithm took: 229.087610 seconds


****************
Run # 6

	Found trivial guesses 21 and 1
	Restarting circuit...


	Non-trivial factors found: 3 and 7

	Algorithm took: 231.016513 seconds


****************
Run